<h3>Imports

In [15]:
from surfalize import Batch

<h3>Get filepaths

In [16]:
from pathlib import Path
root = Path.cwd() / 'matrix'
files = list(root.glob('*.vk4'))

<h3>Create batch and register operations

In [17]:
batch = Batch(files, additional_data='laser_parameters.xlsx')
batch.level()
batch.filter('bandpass', 0.1, 20)
batch.roughness_parameters()

<h3> Execute batch analysis

In [ ]:
df = batch.execute(multiprocessing=True, saveto='results.xlsx')
df

Processing on 12 cores:   0%|          | 0/36 [00:00<?, ?it/s]

<h3>Plot heatmap

In [ ]:
from mplheatmap import heatmap
heatmap(data=df, x='Pulses', y='Fluence', z='depth_mean', ylabel='Fluence (J/cm²)', zlabel='Average structure depth (µm)');

<h3>Other plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Calculate cumulated fluence
df['fcum'] = df['Fluence'] * df['Pulses'] * df['Diameter'] / (df['Hatch factor'] * df['Target Period'])
# Sort values of the dataframe by fcum
df = df.sort_values(by='fcum')

# Create the matplotlib axis and figure
fig, ax = plt.subplots()

# Fitfunction for log fitting
ff = lambda x, a, b, c: a * np.log(b*x) + c
# Perform and plot the log fitting of the data and uncertainty bounds
bounds = ((-np.inf, 0, -np.inf), (np.inf, np.inf, np.inf))

# Logfit data
x = np.linspace(df['fcum'].min(), df['fcum'].max(), 100)
popt, _ = curve_fit(ff, df['fcum'], df['depth_mean'], bounds=bounds)
ax.plot(x, ff(x, *popt), c='tab:blue', ls='-', label='logfit data')
# Logfit upper bound
popt, _ = curve_fit(ff, df['fcum'], df['depth_mean'] + df['depth_std'], bounds=bounds)
yu = ff(x, *popt)
ax.plot(x, yu, c='tab:blue', ls='--')
# Logfit lower bound
popt, _ = curve_fit(ff, df['fcum'], df['depth_mean'] - df['depth_std'], bounds=bounds)
yl = ff(x, *popt)
ax.plot(x, yl, c='tab:blue', ls='--', label='logfit uncertainty')
# Fill blue area
ax.fill_between(x, yl, yu, alpha=0.1)

# Plot the actual data
ax.errorbar(df['fcum'], df['depth_mean'], df['depth_std'], ls='', marker='o', mew=1, mec='k', capsize=3, ecolor='k', elinewidth=1, label='data point')

# Make the plot nicer
ax.grid(ls='--')
ax.set_xlabel('Cumulated fluence (J/cm²)')
ax.set_ylabel('Structure depth (µm)')
ax.legend(loc='lower right')
#fig.savefig('figure.png', dpi=300, bbox_inches='tight')
plt.show()